# corpora

> Functions to work with multiple corpora and download and build sample corpora.
- toc: false
- page-layout: full

In [ ]:
#| default_exp corpora

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
import re
import os
from great_tables import GT
import polars as pl
import msgspec
from fastcore.script import call_parse

In [ ]:
#| export
from conc.core import CorpusMetadata, logger
from conc.corpus import Corpus
from conc.listcorpus import ListCorpus

In [ ]:
#| hide
source_path = f'{os.environ.get("HOME")}/data/'
save_path = f'{os.environ.get("HOME")}/data/conc-test-corpora/'

In [ ]:
#| hide
polars_conf = pl.Config.set_tbl_width_chars(300)
polars_conf = pl.Config.set_fmt_str_lengths(300)

## List available corpora

In [ ]:
#| export
def list_corpora(
		path: str # path to load corpus
		) -> pl.DataFrame: # Dataframe with path, corpus, format (Corpus or List Corpus), corpus name, document count, token count
	""" Scan a directory for available corpora """
	
	available_corpora = {'corpus': [], 'name': [], 'format': [], 'date_created': [], 'document_count': [], 'token_count': []}
	for dir in os.listdir(path):
		if os.path.isdir(os.path.join(path, dir)):
			if os.path.isfile( os.path.join(path, dir, 'corpus.json')):
				with open(os.path.join(path, dir, 'corpus.json'), 'rb') as f:
					data = msgspec.json.decode(f.read(), type=CorpusMetadata)
				available_corpora['format'].append('Corpus')
			elif os.path.isfile( os.path.join(path, dir, 'listcorpus.json')):
				with open(os.path.join(path, dir, 'listcorpus.json'), 'rb') as f:
					data = msgspec.json.decode(f.read(), type=CorpusMetadata)
				available_corpora['format'].append('List Corpus')
			else:
				continue

			available_corpora['corpus'].append(dir)
			for k in ['name', 'document_count', 'token_count', 'date_created']:
				attr = getattr(data, k)
				if isinstance(attr, int):
					attr = f'{attr:,}'
				available_corpora[k].append(attr)

	return pl.DataFrame(available_corpora)

In [ ]:
print(list_corpora(save_path))

┌───────────────────────────┬─────────────────────┬─────────────┬─────────────────────┬────────────────┬─────────────┐
│ corpus                    ┆ name                ┆ format      ┆ date_created        ┆ document_count ┆ token_count │
╞═══════════════════════════╪═════════════════════╪═════════════╪═════════════════════╪════════════════╪═════════════╡
│ bnc.listcorpus            ┆ BNC                 ┆ List Corpus ┆ 2025-07-09 08:54:00 ┆ 4,049          ┆ 113,536,056 │
│ quake-stories-v2.corpus   ┆ Quake Stories v2    ┆ Corpus      ┆ 2025-07-02 09:42:43 ┆ 487            ┆ 472,876     │
│ gutenberg.corpus          ┆ Gutenberg Corpus    ┆ Corpus      ┆ 2025-07-09 09:22:17 ┆ 18             ┆ 2,546,286   │
│ brown.corpus              ┆ Brown Corpus        ┆ Corpus      ┆ 2025-07-09 09:21:45 ┆ 500            ┆ 1,138,566   │
│ brown.listcorpus          ┆ Brown Corpus        ┆ List Corpus ┆ 2025-07-09 09:21:45 ┆ 500            ┆ 1,138,566   │
│ bnc.corpus                ┆ BNC               

## Get data sources

In [ ]:
#| exporti

toy_data = []
toy_data.append(['1.txt', 'The cat sat on the mat.', 'feline', 'cat'])
toy_data.append(['2.txt', 'The dog sat on the mat.', 'canine', 'dog'])
toy_data.append(['3.txt', 'The cat is meowing.', 'feline', 'cat'])
toy_data.append(['4.txt', 'The dog is barking.', 'canine', 'dog'])
toy_data.append(['5.txt', 'The cat is climbing a tree.', 'feline', 'cat'])
toy_data.append(['6.txt', 'The dog is digging a hole.', 'canine', 'dog'])

In [ ]:
#| hide
# checking on counts above
toy_data_test = [doc[1] for doc in toy_data] 
toy_data_test = [re.findall(r'\b\w+\b|[^\w\s]', text) for text in toy_data_test]
toy_data_test = [token.lower() for sublist in toy_data_test for token in sublist if token.strip()]
#print(toy_data_test)
# token count
print(len(toy_data_test)) # should be 38
# word token count
print(len(toy_data_test) - sum([1 for token in toy_data_test if token == '.'])) # should be 32
toy_data_test_unique = set(toy_data_test)
# unique tokens
print(len(toy_data_test_unique))
toy_data_test_unique_word = set([token for token in toy_data_test_unique if token != '.'])
# unique word tokens
print(len(toy_data_test_unique_word))

# based on this - toy corpus should have ... 
# document_count = 6
# token_count = 38
# word_token_count = 32
# unique_tokens = 15
# unique_word_tokens = 14

38
32
15
14


In [ ]:
#| export
def create_toy_corpus_sources(source_path:str # path to location of sources for building corpora
							 ):
	""" Create txt files and csv to test build of toy corpus. """

	toy_path = os.path.join(source_path, 'toy')
	if not os.path.exists(toy_path):
		os.makedirs(toy_path, exist_ok=True)
	for row in toy_data:
		with open(f'{source_path}/toy/{row[0]}', 'w', encoding='utf-8') as f:
			f.write(row[1])
	df = pl.DataFrame(toy_data, orient='row', schema=(('source', str), ('text', str), ('category', str), ('species', str)))
	df.write_csv(os.path.join(source_path, 'toy.csv'))
	df.write_csv(os.path.join(source_path, 'toy.csv.gz'))


In [ ]:
#| hide
#| eval: false
create_toy_corpus_sources(source_path)

In [ ]:
#| export
def show_toy_corpus(
        csv_path:str # path to location of csv for building corpora
        ) -> GT: 
    """ Show toy corpus in a table. """
    
    toy_corpus_df = pl.read_csv(csv_path)
    GT(toy_corpus_df).tab_options(table_margin_left = 0).show()

In [ ]:
show_toy_corpus(os.path.join(source_path, 'toy.csv'))

source,text,category,species
1.txt,The cat sat on the mat.,feline,cat
2.txt,The dog sat on the mat.,canine,dog
3.txt,The cat is meowing.,feline,cat
4.txt,The dog is barking.,canine,dog
5.txt,The cat is climbing a tree.,feline,cat
6.txt,The dog is digging a hole.,canine,dog


In [ ]:
#| export
def get_nltk_corpus_sources(source_path:str # path to location of sources for building corpora
							 ):
	""" Get NLTK corpora as sources for development or testing Conc functionality. """

	try:
		import nltk
	except ImportError as e:
		raise ImportError('This function requires NLTK. To minimise requirements this is not installed by default. You can install NLTK with "pip install nltk"')

	import nltk
	nltk.download('gutenberg')
	nltk.download('brown')
	nltk.download('reuters')
	from nltk.corpus import gutenberg
	from nltk.corpus import reuters
	from nltk.corpus import brown
	import gzip
	import shutil

	def clean_text(text):
		# to match words/punc that followed by /tags
		pattern = re.compile(r"(\S+)(/[^ ]+)") # match non-space followed by / and non-space
		return pattern.sub(r"\1", text)

	if not os.path.exists(source_path):
		os.makedirs(source_path, exist_ok=True)
	if not os.path.exists(f'{source_path}/brown'):
		os.makedirs(f'{source_path}/brown', exist_ok=True)
	brown_path = os.path.join(source_path, 'brown.csv')
	corpus_data = []
	for fileid in brown.fileids():
		corpus_data.append([fileid, clean_text(brown.raw(fileid)), ', '.join(brown.categories(fileid))])
		with open(f'{source_path}/brown/{fileid}.txt', 'w', encoding='utf-8') as f:
			f.write(clean_text(brown.raw(fileid)))
	df = pl.DataFrame(corpus_data, orient='row', schema=(('source', str), ('text', str), ('category', str)))
	df.write_csv(brown_path)

	with open(brown_path, 'rb') as f_in:
		with gzip.open(f'{brown_path}.gz', 'wb') as f_out:
			shutil.copyfileobj(f_in, f_out)

	if os.path.exists(brown_path):
		os.remove(brown_path)

	gutenberg_path = os.path.join(source_path, 'gutenberg.csv')
	corpus_data = []
	for fileid in gutenberg.fileids():
		corpus_data.append([fileid, clean_text(gutenberg.raw(fileid))])
	df = pl.DataFrame(corpus_data, orient='row', schema=(('source', str), ('text', str)))
	df.write_csv(gutenberg_path)

	with open(gutenberg_path, 'rb') as f_in:
		with gzip.open(f'{gutenberg_path}.gz', 'wb') as f_out:
			shutil.copyfileobj(f_in, f_out)

	if os.path.exists(gutenberg_path):
		os.remove(gutenberg_path)

	reuters_path = os.path.join(source_path, 'reuters.csv')
	corpus_data = []
	for fileid in reuters.fileids():
		fileid_name = fileid.split('/')[1]
		corpus_data.append([fileid_name, clean_text(reuters.raw(fileid)), ', '.join(reuters.categories(fileid))])
	df = pl.DataFrame(corpus_data, orient='row', schema=(('source', str), ('text', str), ('categories', str)))
	df.write_csv(reuters_path)

	with open(reuters_path, 'rb') as f_in:
		with gzip.open(f'{reuters_path}.gz', 'wb') as f_out:
			shutil.copyfileobj(f_in, f_out)

	if os.path.exists(reuters_path):
		os.remove(reuters_path)


In [ ]:
#| hide
#| eval: false
get_nltk_corpus_sources(source_path)

[nltk_data] Downloading package gutenberg to /home/geoff/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package brown to /home/geoff/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package reuters to /home/geoff/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


The texts for the Brown corpus from nltk can be used to test Conc functionality. The Reuters and Gutenberg corpora are also prepared by `get_nltk_corpus_sources`. Running the function will download the texts and save the texts as a .csv.gz files with columns: source and text. The Brown Corpus is also saved as .txt files to test the Corpus.build_from_texts method.

In [ ]:
#| export
def parse_bnc_to_csv(source_path:str, # path to location of sources for building corpora, this is like to be a path ending with 'Texts'
					 save_path:str, # path to save the csv
					 output_filename:str = 'bnc.csv.gz' # name of the output file e.g. bnc.csv.gz or bnc-baby.csv.gz
					):
	""" Converts BNC XML files, available via the British National Corpus, XML edition and the British National Corpus, Baby edition to a compressed CSV with title information retained. """

	import os
	import gzip
	import csv
	from lxml import etree
	from glob import glob

	def extract_text_and_title_from_bnc_xml(xml_path: str # path to the XML file
										 ) -> tuple[str, str]: # title, text, bibliographic data, keywords, classcode, catref_targets
		""" Extract title (from sourceDesc) and text via sentences from a BNC XML file. """

		tree = etree.parse(xml_path)

		# print(etree.tostring(tree.find('.//teiHeader'), pretty_print=True, encoding='unicode'))
		title_el = tree.find('.//title')
		title = title_el.text.strip() if title_el is not None and title_el.text else ''

		bibl_el = tree.find('.//bibl')
		if bibl_el is None:
			logger.debug(f'No bibl element found in {xml_path}. This may not be a valid BNC XML file.')
			bibliographic_data = ''
		else:
			bibliographic_data = ''.join(bibl_el.itertext()) if bibl_el is not None else ''
		profiledesc_el = tree.find('.//profileDesc')
		if profiledesc_el is None:
			logger.debug(f'No profileDesc found in {xml_path}. This may not be a valid BNC XML file.')
			keywords, classcode, catref_targets = '', '', ''
		else:
			keywords_el = profiledesc_el.find('.//keywords')
			keywords = ''.join(keywords_el.itertext()) if keywords_el is not None else ''
			if keywords.strip() == '(none)':
				keywords = ''
			classcode_el = profiledesc_el.find('.//classCode')
			classcode = ''.join(classcode_el.itertext()) if classcode_el is not None else ''
			catref_el = profiledesc_el.find('.//catRef')
			catref_targets = ''
			if catref_el is not None:
				catref_targets = catref_el.attrib.get('targets', '') 

		sentences = []
		for s in tree.xpath('.//s'):
			tokens = [t.text for t in s.iter() if t.text and t.text.strip()]
			if tokens:
				sentences.append(''.join(tokens))
		if len(sentences) == 0:
			logger.warning(f'No sentences found in {xml_path}. This may not be a valid BNC XML file.')

		return title, '\n'.join(sentences), bibliographic_data, keywords, classcode, catref_targets

	output_path = os.path.join(save_path, output_filename)
	xml_files = glob(os.path.join(source_path, '**', '*.xml'), recursive=True)
	os.makedirs(save_path, exist_ok=True)

	with gzip.open(output_path, 'wt', encoding='utf-8', newline='') as csvfile:
		writer = csv.writer(csvfile)
		writer.writerow(['source', 'title', 'bibliographic_data', 'keywords', 'classcode', 'catref_targets', 'text'])

		for xml_file in xml_files:
			try:
				title, text, bibliographic_data, keywords, classcode, catref_targets = extract_text_and_title_from_bnc_xml(xml_file)
				writer.writerow([os.path.relpath(xml_file, source_path), title, bibliographic_data, keywords, classcode, catref_targets, text])
			except Exception as e:
				logger.warning(f"Error processing {xml_file}: {e}")

	

In [ ]:
#| hide
# EXPERIMENTING WITH GETTING METADATA FROM BNC - teiheader contains metadata both in text and attributes
# very rough parsing below to json
# ALSO  - see http://www.natcorp.ox.ac.uk/archive/worldURG/urg.pdf
# content of each text is marked up with paragraphs etc - consider reformatting using these tags to get nicer flowing text
# see sections on spoken and written texts

# from nltk.corpus.reader.bnc import BNCCorpusReader
# from bs4 import BeautifulSoup
# import json

# def bs4_to_dict(tag):
#     node = {}
#     # Add attributes
#     if tag.attrs:
#         node.update(('@' + k, v) for k, v in tag.attrs.items())
#     # Add children or text
#     children = [child for child in tag.children if child.name]
#     if children:
#         for child in children:
#             child_dict = bs4_to_dict(child)
#             if child.name in node:
#                 if not isinstance(node[child.name], list):
#                     node[child.name] = [node[child.name]]
#                 node[child.name].append(child_dict)
#             else:
#                 node[child.name] = child_dict
#     if tag.string and tag.string.strip():
#         node['#text'] = tag.string.strip()
#     return node

# path_to_bnc_texts = f'{source_path}bnc-baby-xml/download/Texts'
# corpus = BNCCorpusReader(root=path_to_bnc_texts, fileids=r'.*\.xml')

# i = 0
# for fileid in corpus.fileids():
#     # open using lxml
#     print(f'Processing {fileid}')
#     raw_xml = open(os.path.join(path_to_bnc_texts, fileid), 'r', encoding='utf-8').read()

#     soup = BeautifulSoup(raw_xml, 'lxml')
#     item = soup.find('teiheader')

#     item_dict = {item.name: bs4_to_dict(item)}
#     print(json.dumps(item_dict, indent=2))


#     break



The 1994 version of the British National Corpus (BNC) is available from the [Oxford Text Archive](https://llds.ling-phil.ox.ac.uk/llds/xmlui/discover?filtertype=author&filter_relational_operator=equals&filter=BNC+Consortium). The `parse_bnc_to_csv` function assumes you have downloaded and unziped the files. You need to specify the directory containing the XML files for the texts. This will be a path ending with `Texts`. Conc does not provide a way to download the BNC zip files directly. You need to go to the Oxford Text Archive and read the notes about restricted use. Note: this function was previously using NLTK's BNC parser, but this has been rewritten for lxml, which is much (2x) faster. 

In [ ]:
#| hide
#| eval: false
# LEAVE COMMENTED OUT FOR CI AS BNC NOT AVAILABLE - takes about 40 seconds to run
# parse_bnc_to_csv(source_path = f'{source_path}bnc-baby-xml/download/Texts', save_path = source_path, output_filename = 'bnc-baby.csv.gz')
# pl.scan_csv(os.path.join(source_path, 'bnc-baby.csv.gz')).head(2).collect()

In [ ]:
#| hide
#| eval: false
# LEAVE COMMENTED OUT FOR CI AS BNC NOT AVAILABLE 
# parse_bnc_to_csv(source_path = f'{source_path}bnc-xml/download/Texts', save_path = source_path, output_filename = 'bnc.csv.gz')
# pl.scan_csv(os.path.join(source_path, 'bnc.csv.gz')).head(2).select().collect()

In [ ]:
#| hide
#| eval: false
# LEAVE COMMENTED OUT FOR CI AS BNC NOT AVAILABLE - takes about 40 seconds to run
# corpus_name = 'Baby BNC'
# corpus_description = 'British National Corpus, Baby edition. Please use the following text to cite this item or export to a predefined format: BNC Consortium, 2007, British National Corpus, Baby edition, Literary and Linguistic Data Service, http://hdl.handle.net/20.500.14106/2553.'
# metadata_columns = ['source', 'title']
# corpus = Corpus(name = corpus_name, description = corpus_description).build_from_csv(source_path = f'{source_path}bnc-baby.csv.gz', text_column='text', metadata_columns=metadata_columns, save_path = save_path, standardize_word_token_punctuation_characters = True)
# corpus.summary()

In [ ]:
#| hide
#| eval: false
# LEAVE COMMENTED OUT FOR CI AS BNC NOT AVAILABLE - takes about 10 minutes to run
# corpus_name = 'BNC'
# corpus_description = 'British National Corpus, XML edition. Please use the following text to cite this item or export to a predefined format: BNC Consortium, 2007, British National Corpus, XML edition, Literary and Linguistic Data Service, http://hdl.handle.net/20.500.14106/2554. '
# metadata_columns = ['source']
# corpus = Corpus(name = corpus_name, description = corpus_description).build_from_csv(source_path = f'{source_path}bnc.csv.gz', text_column='text', metadata_columns=metadata_columns, save_path = save_path, standardize_word_token_punctuation_characters = True)
# corpus.summary()

In [ ]:
#| hide
#| eval: false
# LEAVE COMMENTED OUT FOR CI AS BNC NOT AVAILABLE
# listcorpus = ListCorpus().build_from_corpus(source_corpus_path = f'{save_path}bnc.corpus', save_path = save_path)
# listcorpus = ListCorpus().build_from_corpus(source_corpus_path = f'{save_path}baby-bnc.corpus', save_path = save_path)

In [ ]:
#| export
def get_garden_party(source_path: str, #path to location of sources for building corpora
					 create_archive_variations: bool = False # create .tar and .tar.gz files for dev/testing (leave False if you just want the zip)
					):
	""" Get corpus of The Garden Party by Katherine Mansfield for development of Conc and testing Conc functionality. """

	path = 'https://github.com/ucdh/scraping-garden-party/raw/master/garden-party-corpus.zip'

	import requests
	try:
		import requests
	except ImportError as e:
		raise ImportError('This function requires the requests library. To minimise requirements this is not installed by default. You can install requests with "pip install requests"')

	if not os.path.exists(source_path):
		os.makedirs(source_path)

	r = requests.get(path)
	with open(f'{source_path}/garden-party-corpus.zip', 'wb') as f:
		f.write(r.content)

	if create_archive_variations: 	# converting to .tar and tar.gz files for testing
		import zipfile
		with zipfile.ZipFile(f'{source_path}/garden-party-corpus.zip', 'r') as z:
			z.extractall(f'{source_path}/garden-party-corpus')
		import shutil # make tar.gz
		shutil.make_archive(f'{source_path}/garden-party-corpus', 'gztar', f'{source_path}/garden-party-corpus')
		shutil.make_archive(f'{source_path}/garden-party-corpus', 'tar', f'{source_path}/garden-party-corpus')
		shutil.rmtree(f'{source_path}/garden-party-corpus')
	

The `get_garden_party` function downloads a zip file of an example corpus based on Katherine Mansfield short stories. This function creates a .tar and a .tar.gz version of the texts for testing Corpus build methods.  

In [ ]:
#| eval: false
get_garden_party(source_path, create_archive_variations = True)

## Create large corpora for development and testing

In [ ]:
#| export
def get_large_dataset(source_path: str #path to location of sources for building corpora
					):
	""" Get 1m rows of https://huggingface.co/datasets/Eugleo/us-congressional-speeches-subset for testing. """
	import gzip
	import shutil
	
	path = f'{source_path}/us-congressional-speeches-subset-1m.csv'
	df = pl.read_parquet('hf://datasets/Eugleo/us-congressional-speeches-subset/data/train-*.parquet')
	df.sample(1000000).select(['speech_id', 'date', 'speaker', 'chamber', 'state', 'text']).write_csv(path)
	del df

	with open(path, 'rb') as f_in:
		with gzip.open(f'{path}.gz', 'wb') as f_out:
			shutil.copyfileobj(f_in, f_out)

	if os.path.exists(path):
		os.remove(path)


In [ ]:
#| hide
# get_large_dataset(source_path)

In [ ]:
#| hide
# Define the chunk size
# chunk_size = 100_000  # Adjust based on your memory constraints

# # Lazily load the CSV file
# df = pl.scan_csv(f'{source_path}us-congressional-speeches-subset-1m.csv.gz')

# # Add the new column 'is_empty'
# df = df.with_columns(
#     (pl.col('text').str.strip_chars().eq('')).alias('is_empty')
# )

# # get length of is_empty where True
# count = df.filter(pl.col("is_empty") == True).collect().height
# print(f"Number of empty rows: {count}")

# any empty?
#len(df[df['text'].is_null()])

# get distribution of date (by year), speaker, chamber, state
# dates are in iso format - extract year and summarize
# df = df.with_columns(pl.col('date').str.slice(0, 4).alias('year'))
# df.group_by('year').agg(pl.count('year').alias('count')).sort('year', descending=True).head(10).collect()

# #df.group_by('speaker').agg(pl.count('speaker').alias('count')).sort('count', descending=True).head(20)
# #df.group_by('chamber').agg(pl.count('chamber').alias('count')).sort('count', descending=True).head(20)
# df.group_by('state').agg(pl.count('state').alias('count')).sort('count', descending=True).head(20)

In [ ]:
#| export
def create_large_dataset_sizes(source_path: str, #path to location of sources for building corpora
						sizes: list = [10000, 100000, 200000, 500000] # list of sizes for test data-sets
						):
	""" Create datasets of different sizes from data source retrieved by get_large_dataset for testing. """
	import gzip
	import shutil

	for max_i in sizes:
		max_i_label = int(max_i / 1000)
		path = f'{source_path}/us-congressional-speeches-subset-{max_i_label}k.csv'
		df = pl.read_csv(f'{source_path}/us-congressional-speeches-subset-1m.csv.gz')
		df.sample(max_i).write_csv(path)
		logger.info(f'Creating dataset of {max_i_label}k rows')

		with open(path, 'rb') as f_in:
			with gzip.open(f'{path}.gz', 'wb') as f_out:
				shutil.copyfileobj(f_in, f_out)

		if os.path.exists(path):
			os.remove(path)


In [ ]:
#| hide
# create_large_dataset_sizes(source_path)

## Build sample corpora



In [ ]:
#| export
@call_parse
def build_sample_corpora(
		source_path:str, # path to folder with corpora
		save_path:str, # path to save corpora
		force_rebuild:bool = False # force rebuild of corpora, useful for development and testing
		):
	"""Build all test corpora from source files."""

	if force_rebuild:
		import shutil

	corpora = {}
	corpora['toy'] = {'name': 'Toy Corpus', 'slug': 'toy', 'description': 'Toy corpus is a very small dataset for testing and library development. ', 'extension': '.csv.gz', 'metadata_columns': ['source', 'category', 'species']}
	corpora['brown'] = {'name': 'Brown Corpus', 'slug': 'brown', 'description': 'A Standard Corpus of Present-Day Edited American English, for use with Digital Computers. by W. N. Francis and H. Kucera (1964) Department of Linguistics, Brown University Providence, Rhode Island, USA Revised 1971, Revised and Amplified 1979 http://www.hit.uib.no/icame/brown/bcm.html. This version downloaded via NLTK https://www.nltk.org/nltk_data/.', 'extension': '.csv.gz', 'metadata_columns': ['source', 'category']}
	corpora['reuters'] = {'name': 'Reuters Corpus', 'slug': 'reuters', 'description': 'Reuters corpus (Reuters-21578, Distribution 1.0). "The copyright for the text of newswire articles and Reuters annotations in the Reuters-21578 collection resides with Reuters Ltd. Reuters Ltd. and Carnegie Group, Inc. have agreed to allow the free distribution of this data *for research purposes only*. If you publish results based on this data set, please acknowledge its use, refer to the data set by the name (Reuters-21578, Distribution 1.0), and inform your readers of the current location of the data set." https://kdd.ics.uci.edu/databases/reuters21578/reuters21578.html. This version downloaded via NLTK https://www.nltk.org/nltk_data/.', 'extension': '.csv.gz', 'metadata_columns': ['source', 'categories']}
	corpora['gutenberg'] = {'name': 'Gutenberg Corpus', 'slug': 'gutenberg', 'description': 'Project Gutenberg Selections NLTK Corpus. Source: https://gutenberg.org/. Public domain. This version downloaded via NLTK https://www.nltk.org/nltk_data/.', 'extension': '.csv.gz', 'metadata_columns': ['source']}
	corpora['garden-party-corpus'] = {'name': 'Garden Party Corpus', 'slug': 'garden-party', 'description': 'A corpus of short stories from The Garden Party: and Other Stories by Katherine Mansfield. Texts downloaded from Project Gutenberg https://gutenberg.org/ and are in the public domain. The text files contain the short story without the title. https://github.com/ucdh/scraping-garden-party', 'extension': '.zip', 'metadata_columns': None}

	if not os.path.exists(source_path):
		os.makedirs(source_path)
	if not os.path.exists(save_path):
		os.makedirs(save_path)

	if not os.path.exists(f'{source_path}toy.csv.gz'):
		create_toy_corpus_sources(source_path)

	if not os.path.exists(f'{source_path}garden-party-corpus.zip'):
		get_garden_party(source_path)

	if not os.path.exists(f'{source_path}brown.csv.gz'):
		get_nltk_corpus_sources(source_path)

	for corpus_name, corpus_details in corpora.items():
		if force_rebuild and os.path.isdir(f'{save_path}{corpus_details["slug"]}.corpus'):
			shutil.rmtree(f'{save_path}{corpus_details["slug"]}.corpus', ignore_errors=True)

		try:
			corpus = Corpus().load(f"{save_path}{corpus_details['slug']}.corpus")
		except FileNotFoundError:

			if 'csv' in corpus_details['extension']:
				corpus = Corpus(name = corpus_details['name'], description = corpus_details['description']).build_from_csv(source_path = f'{source_path}{corpus_name}.csv.gz', text_column='text', metadata_columns=corpus_details['metadata_columns'], save_path = save_path)
			else:
				corpus = Corpus(name = corpus_details['name'], description = corpus_details['description']).build_from_files(source_path = f'{source_path}{corpus_name}{corpus_details["extension"]}', save_path = save_path)
		except Exception as e:
			raise e
		del corpus
	
	if force_rebuild and os.path.isdir(f'{save_path}brown.listcorpus'):
		shutil.rmtree(f'{save_path}brown.listcorpus', ignore_errors=True)
	try:
		listcorpus = ListCorpus().load(f"{save_path}brown.listcorpus")
	except FileNotFoundError:
		listcorpus = ListCorpus().build_from_corpus(source_corpus_path = f'{save_path}brown.corpus', save_path = save_path)
	del listcorpus

The `build_sample_corpora` function downloads sources and creates sample corpora for development and testing for releases. These datasets are a good way to get started working with Conc. Sample corpora available are:

* Brown Corpus (via NLTK)  
* Gutenberg Corpus (via NLTK)  
* Reuters Corpus (via NLTK)  
* Garden Party Corpus (Katherine Mansfield short stories)  
* Toy Corpus (6 very short texts for testing only)  
* Brown List Corpus (a lightweight ListCorpus version of the Brown Corpus for use with keywords functionality)

After installing Conc you can invoke this function from the command line to download and build the sample corpora:
```sh
conc_build_sample_corpora path/to/save/sources path/to/save/corpora
```


Note: `build_sample_corpora` was accessible via conc.corpus as `build_test_corpora` up to version 0.1.1. This functionality is only accessible from `conc.corpora` now.  

In [ ]:
#| hide
from conc.core import set_logger_state

In [ ]:
build_sample_corpora(source_path, save_path, force_rebuild=False) # must be left as False after dev, otherwise could destroy corpora mid test in CI

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()